In [32]:
from lxml import etree
import pandas as pd
import requests
import re

url = "http://211.184.196.130/rest/JejuSewerWaterQDataService/getJejuSewerWaterQData/"
response = requests.get(url)

if response.status_code == 200:
    xml_data = response.text

    # 제어 문자를 제거
    cleaned_xml_data = re.sub(r'[\x00-\x1F\x7F]', '', xml_data)

    try:
        # lxml을 사용한 파싱
        root = etree.fromstring(cleaned_xml_data)

        # 데이터를 저장할 리스트
        data = []

        # 모든 <list> 태그를 찾아서 반복
        for list_item in root.findall('.//list'):
            item_data = {
                'PlaceCode': list_item.find('PlaceCode').text,
                'MeasureTime': list_item.find('MeasureTime').text,
                'Organic': list_item.find('Organic').text,
                'Organic_Status': list_item.find('Organic_Status').text,
                'Suspended': list_item.find('Suspended').text,
                'Suspended_Status': list_item.find('Suspended_Status').text,
                'TotalP': list_item.find('TotalP').text,
                'TotalP_Status': list_item.find('TotalP_Status').text,
                'TotalN': list_item.find('TotalN').text,
                'TotalN_Status': list_item.find('TotalN_Status').text,
                'CommDisorder': list_item.find('CommDisorder').text
            }
            data.append(item_data)

        # DataFrame으로 변환
        df = pd.DataFrame(data)
        df['MeasureTime'] = df['MeasureTime'][0]

        # 결과 출력
        print(df)

    except etree.XMLSyntaxError as e:
        print("XMLSyntaxError:", e)
    except Exception as e:
        print("Unexpected error:", e)

  PlaceCode     MeasureTime Organic Organic_Status Suspended Suspended_Status  \
0    Bo-Mok  20240904234210   2.5               00     0.8                 00   
1  Dae-Jung  20240904234210   5.8               00     1.1                 00   
2  Eastside  20240904234210   8.4               00     5.1                 00   
3      Jeju  20240904234210   4.7               00     2.6                 00   
4   Nam-Won  20240904234210   0.0               04    2.1                  00   
5   Sek-Dal  20240904234210   6.4               00     2.8                 00   
6  Sung-San  20240904234210    None           None      None             None   
7  Westside  20240904234210   6.9               00     2.0                 00   

   TotalP TotalP_Status   TotalN TotalN_Status CommDisorder  
0  1.390             00  10.962             00           OK  
1  1.005             00  3.238              00           OK  
2  0.746             00  11.966             00           NG  
3  0.704             0

In [37]:
df.columns = ['하수처리장코드', '측정일시', '화학적산소요구량', '화학적산소요구량 측정상태값',
              '부유물질농도', '부유물질농도 측정상태값', '총인(P)농도', '총 인(P)농도 측정상태값', 
              '총 질소(N)농도', '총 질소(N)농도 측정상태값', '통신이상상태']

,하수처리장코드,측정일시,화학적산소요구량,화학적산소요구량 측정상태값,부유물질농도,부유물질농도 측정상태값,총인(P)농도,총 인(P)농도 측정상태값,총 질소(N)농도,총 질소(N)농도 측정상태값,통신이상상태
0,Bo-Mok,20240904234210,2.5,00,0.8,00,1.390,00,10.962,00,OK
1,Dae-Jung,20240904234210,5.8,00,1.1,00,1.005,00,3.238,00,OK
2,Eastside,20240904234210,8.4,00,5.1,00,0.746,00,11.966,00,NG
3,Jeju,20240904234210,4.7,00,2.6,00,0.704,00,8.313,00,NG
4,Nam-Won,20240904234210,0.0,04,2.1,00,1.327,00,5.567,00,OK
5,Sek-Dal,20240904234210,6.4,00,2.8,00,1.672,00,9.793,00,NG
6,Sung-San,20240904234210,None,None,None,None,None,None,None,None,NG
7,Westside,20240904234210,6.9,00,2.0,00,1.001,00,4.557,00,OK


In [49]:
df['총인(P)농도'] = df['총인(P)농도'].astype('float')
df['총 질소(N)농도'] = df['총 질소(N)농도'].astype('float')
df['화학적산소요구량'] = df['화학적산소요구량'].astype('float')
df['부유물질농도'] = df['부유물질농도'].astype('float')

In [68]:
import pandas as pd

# 수질 기준 설정
def evaluate_water_quality(row):
    # 부영양화 평가
    if row.isnull().any():
        eutrophication_grade = '통신이상'
    elif row['총인(P)농도'] > 0.02 or row['총 질소(N)농도'] > 0.3:
        eutrophication_grade = '나쁨'
    else:
        eutrophication_grade = '양호'

    # 유기물 오염 평가
    if row.isnull().any():
        organic_pollution_grade = '통신이상'
    elif row['화학적산소요구량'] > 5:
        organic_pollution_grade = '나쁨'
    else:
        organic_pollution_grade = '양호'

    # 물리적 오염 평가
    if row.isnull().any():
        physical_pollution_grade = '통신이상'
    elif row['부유물질농도'] > 25:
        physical_pollution_grade = '나쁨'
    else:
        physical_pollution_grade = '양호'

    # 전체 수질 등급 종합 (간단한 합산 방식 사용)
    if row.isnull().any():
        overall_grade = '통신이상'
    elif '나쁨' in [eutrophication_grade, organic_pollution_grade, physical_pollution_grade]:
        overall_grade = '나쁨'
    else:
        overall_grade = '양호'

    return pd.Series([eutrophication_grade, organic_pollution_grade, physical_pollution_grade, overall_grade],
                     index=['부영양화 등급', '유기물 오염 등급', '물리적 오염 등급', '종합 수질 등급'])

# 수질 평가 적용
df[['부영양화 등급', '유기물 오염 등급', '물리적 오염 등급', '종합 수질 등급']] = df.apply(evaluate_water_quality, axis=1)

# 결과 출력
print(df[['하수처리장코드', '부영양화 등급', '유기물 오염 등급', '물리적 오염 등급', '종합 수질 등급']])

    하수처리장코드 부영양화 등급 유기물 오염 등급 물리적 오염 등급 종합 수질 등급
0    Bo-Mok      나쁨        양호        양호       나쁨
1  Dae-Jung      나쁨        나쁨        양호       나쁨
2  Eastside      나쁨        나쁨        양호       나쁨
3      Jeju      나쁨        양호        양호       나쁨
4   Nam-Won      나쁨        양호        양호       나쁨
5   Sek-Dal      나쁨        나쁨        양호       나쁨
6  Sung-San    통신이상      통신이상      통신이상     통신이상
7  Westside      나쁨        나쁨        양호       나쁨


In [69]:
# 하천수와 호소수 기준

import pandas as pd

def evaluate_river_quality(row):
    # 하천수 기준에 따라 평가
    if row.isnull().any():
        return '통신이상'
    elif row['화학적산소요구량'] <= 2 and row['총인(P)농도'] <= 0.02 and row['부유물질농도'] <= 25:
        return "Ia"
    elif row['화학적산소요구량'] <= 4 and row['총인(P)농도'] <= 0.04 and row['부유물질농도'] <= 25:
        return "Ib"
    elif row['화학적산소요구량'] <= 5 and row['총인(P)농도'] <= 0.1 and row['부유물질농도'] <= 25:
        return "II"
    elif row['화학적산소요구량'] <= 7 and row['총인(P)농도'] <= 0.2 and row['부유물질농도'] <= 25:
        return "III"
    elif row['화학적산소요구량'] <= 9 and row['총인(P)농도'] <= 0.3 and row['부유물질농도'] <= 100:
        return "IV"
    elif row['화학적산소요구량'] <= 11 and row['총인(P)농도'] <= 0.5:
        return "V"
    else:
        return "VI"

def evaluate_lake_quality(row):
    # 호소수 기준에 따라 평가
    if row.isnull().any():
        return '통신이상'
    elif row['화학적산소요구량'] <= 2 and row['총인(P)농도'] <= 0.01 and row['부유물질농도'] <= 1:
        return "Ia"
    elif row['화학적산소요구량'] <= 3 and row['총인(P)농도'] <= 0.02 and row['부유물질농도'] <= 5:
        return "Ib"
    elif row['화학적산소요구량'] <= 4 and row['총인(P)농도'] <= 0.03 and row['부유물질농도'] <= 5:
        return "II"
    elif row['화학적산소요구량'] <= 5 and row['총인(P)농도'] <= 0.05 and row['부유물질농도'] <= 15:
        return "III"
    elif row['화학적산소요구량'] <= 8 and row['총인(P)농도'] <= 0.1 and row['부유물질농도'] <= 15:
        return "IV"
    elif row['화학적산소요구량'] <= 10 and row['총인(P)농도'] <= 0.15:
        return "V"
    else:
        return "VI"


# 하천수 등급 평가
df['하천수 등급'] = df.apply(evaluate_river_quality, axis=1)

# 호소수 등급 평가
df['호소수 등급'] = df.apply(evaluate_lake_quality, axis=1)

df

,하수처리장코드,측정일시,화학적산소요구량,화학적산소요구량 측정상태값,부유물질농도,부유물질농도 측정상태값,총인(P)농도,총 인(P)농도 측정상태값,총 질소(N)농도,총 질소(N)농도 측정상태값,통신이상상태,부영양화 등급,유기물 오염 등급,물리적 오염 등급,종합 수질 등급,하천수 등급,호소수 등급,방류 가능 여부
0,Bo-Mok,20240904234210,2.5,00,0.8,00,1.390,00,10.962,00,OK,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
1,Dae-Jung,20240904234210,5.8,00,1.1,00,1.005,00,3.238,00,OK,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
2,Eastside,20240904234210,8.4,00,5.1,00,0.746,00,11.966,00,NG,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
3,Jeju,20240904234210,4.7,00,2.6,00,0.704,00,8.313,00,NG,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
4,Nam-Won,20240904234210,0.0,04,2.1,00,1.327,00,5.567,00,OK,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
5,Sek-Dal,20240904234210,6.4,00,2.8,00,1.672,00,9.793,00,NG,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
6,Sung-San,20240904234210,NaN,None,NaN,None,NaN,None,NaN,None,NG,통신이상,통신이상,통신이상,통신이상,통신이상,통신이상,통신이상
7,Westside,20240904234210,6.9,00,2.0,00,1.001,00,4.557,00,OK,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능


In [70]:
# 방류수 수질기준 설정 (2013.1.1 이후 기준)
criteria = {
    '화학적산소요구량': 40,  # COD
    '부유물질농도': 10,  # SS
    '총인(P)농도': 0.5,  # T-P
    '총 질소(N)농도': 20  # T-N
}

# 방류 가능 여부를 평가하는 함수
def evaluate_lake_quality(row):
    if row.isnull().any():
        return '통신이상'
    else:
        for col, limit in criteria.items():
            if row[col] > limit:
                return "방류 불가능"
        return "방류 가능"

# 각 행에 대해 방류 가능 여부를 평가하여 새로운 컬럼 추가
df['방류 가능 여부'] = df.apply(evaluate_lake_quality, axis=1)
df


,하수처리장코드,측정일시,화학적산소요구량,화학적산소요구량 측정상태값,부유물질농도,부유물질농도 측정상태값,총인(P)농도,총 인(P)농도 측정상태값,총 질소(N)농도,총 질소(N)농도 측정상태값,통신이상상태,부영양화 등급,유기물 오염 등급,물리적 오염 등급,종합 수질 등급,하천수 등급,호소수 등급,방류 가능 여부
0,Bo-Mok,20240904234210,2.5,00,0.8,00,1.390,00,10.962,00,OK,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
1,Dae-Jung,20240904234210,5.8,00,1.1,00,1.005,00,3.238,00,OK,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
2,Eastside,20240904234210,8.4,00,5.1,00,0.746,00,11.966,00,NG,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
3,Jeju,20240904234210,4.7,00,2.6,00,0.704,00,8.313,00,NG,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
4,Nam-Won,20240904234210,0.0,04,2.1,00,1.327,00,5.567,00,OK,나쁨,양호,양호,나쁨,VI,VI,방류 불가능
5,Sek-Dal,20240904234210,6.4,00,2.8,00,1.672,00,9.793,00,NG,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능
6,Sung-San,20240904234210,NaN,None,NaN,None,NaN,None,NaN,None,NG,통신이상,통신이상,통신이상,통신이상,통신이상,통신이상,통신이상
7,Westside,20240904234210,6.9,00,2.0,00,1.001,00,4.557,00,OK,나쁨,나쁨,양호,나쁨,VI,VI,방류 불가능


In [27]:
!pip install geopy

In [28]:
from geopy.geocoders import Nominatim
import pandas as pd

geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

address = pd.DataFrame({
    'name' : ['Jeju', 'Eastside', 'Sek-Dal', 'Bo-Mok', 'Dae-Jung', 'Sung-San', 'Westside', 'Nam-Won'], 
    'address' : ['제주시 도리로 254', '제주특별자치도 제주시 한경면 일주서로 4641', '서귀포시 예래해안로 348',
                 '제주특별자치도 서귀포시 보목로64번길 155', '제주특별자치도 서귀포시 대정읍 최남단해안로302번길 37', 
                 '서귀포시 고성동서로 115', '제주특별자치도 제주시 구좌읍 월정리 1544-12', '제주특별자치도 서귀포시 남원읍 태흥리 73-1']
})
    
latitude = []
longitude =[]

for i in range(len(address)):
    latitude.append(geocoding(address.iloc[i]['address'])[0])
    longitude.append(geocoding(address.iloc[i]['address'])[1])

address['latitude']=latitude
address['longitude']=longitude

In [29]:
address['latitude'][6]= 33.562058
address['longitude'][6] = 126.779656
address['latitude'][7] = 33.288276
address['longitude'][7] = 126.754429
address

/var/folders/47/vl2hvgw96391mk95fsdgkblh0000gn/T/ipykernel_1917/3388394425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address['latitude'][6]= 33.562058
/var/folders/47/vl2hvgw96391mk95fsdgkblh0000gn/T/ipykernel_1917/3388394425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address['longitude'][6] = 126.779656
/var/folders/47/vl2hvgw96391mk95fsdgkblh0000gn/T/ipykernel_1917/3388394425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,name,address,latitude,longitude
0,Jeju,제주시 도리로 254,33.508016,126.476287
1,Eastside,제주특별자치도 제주시 한경면 일주서로 4641,33.320554,126.181241
2,Sek-Dal,서귀포시 예래해안로 348,33.242818,126.392671
3,Bo-Mok,제주특별자치도 서귀포시 보목로64번길 155,33.239640,126.593145
4,Dae-Jung,제주특별자치도 서귀포시 대정읍 최남단해안로302번길 37,33.198641,126.266687
5,Sung-San,서귀포시 고성동서로 115,33.451035,126.919956
6,Westside,제주특별자치도 제주시 구좌읍 월정리 1544-12,33.562058,126.779656
7,Nam-Won,제주특별자치도 서귀포시 남원읍 태흥리 73-1,33.288276,126.754429


In [81]:
address = address.sort_values('name')

In [82]:
df = df.sort_values('하수처리장코드')

In [83]:
# 팝업 텍스트를 위한 함수
def create_popup(row):
    popup_content = f'''
    <div style="width:200px">
        <strong>하수처리장코드:</strong> {row['하수처리장코드']}<br>
        <strong>통신이상상태:</strong> {row['통신이상상태']}<br>
        <strong>부영양화 등급:</strong> {row['부영양화 등급']}<br>
        <strong>유기물 오염 등급:</strong> {row['유기물 오염 등급']}<br>
        <strong>물리적 오염 등급:</strong> {row['물리적 오염 등급']}<br>
        <strong>종합 수질 등급:</strong> {row['종합 수질 등급']}<br>
        <strong>하천수 등급:</strong> {row['하천수 등급']}<br>
        <strong>호소수 등급:</strong> {row['호소수 등급']}<br>
        <strong>방류 가능 여부:</strong> {row['방류 가능 여부']}
    </div>
    '''
    return popup_content

In [84]:
import folium

map_ = folium.Map(
    location = [33.3684955195788, 126.52918183373025],
    #tiles = 'Stamen Terrain', # 지형을 고려한 지도
    zoom_start = 10
)

tooltip = 'Click!!'

for i in range(len(address)):
    folium.Marker(location = [address.iloc[i]['latitude'],address.iloc[i]['longitude']],
                  popup=create_popup(df.iloc[i]),
                  tooltip = tooltip,
                  icon = folium.Icon(color = 'red', icon='info-sign') 
    ).add_to(map_)
    
map_